In [1]:
import petl as etl
from utils.paths import PROCESSED, SITE

In [2]:
WARD = SITE / 'insights/reach/_data'
WARD.mkdir(exist_ok=True, parents=True)

In [3]:
wards = etl.fromcsv(SITE / '_data/wards.csv')

## Tickets

In [4]:
tickets = (
    etl
    .fromcsv(PROCESSED / 'ticketing/tickets.csv')
    .selecteq('geography_type', 'osward')
    .selectne('geography_code', 'OTHER')
    .convertnumbers()
    # TODO May need to filter out specific ticket types
    .aggregate(['geography_code'], sum, 'count_of_tickets')
    .addfield('variable', 'tickets', index=1)
)

total_bradford_sales = tickets.values('value').sum()

tickets_prop = (
    tickets
    .convert('value', lambda x: x / total_bradford_sales)
    .convert('value', lambda x: round(x, 3))
    .update('variable', 'tickets_prop')
)

## Surveys

In [7]:
surveys = (
    etl
    .fromcsv(PROCESSED / 'surveys/respondents.csv')
    .cutout('geography_type')
    .convertnumbers()
)

## Combine ward stats

In [8]:
data = (
    etl
    .cat(
        tickets,
        tickets_prop,
        surveys,
    )
    .recast(['geography_code'])
    .leftjoin(
        wards.cut('code', 'name'),
        lkey='geography_code',
        rkey='code'
    )
    .sort('name')
)

In [9]:
data.tojson(WARD / 'by_ward.json', indent=2)